## 키워드 검색량 도구 API test

In [1]:
import random
import urllib
import time
from datetime import datetime 
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import json
import signaturehelper
import requests

In [2]:
config_secret_debug = json.loads(open(".config_secret/settings_debug.json").read())
customer_id = config_secret_debug["NAVER_SEARCH"]["CUSTOMER_ID"]
client_id = config_secret_debug["NAVER_SEARCH"]["CLIENT_ID"]
client_secret = config_secret_debug["NAVER_SEARCH"]["CLIENT_SECRET"]

In [80]:
## 입력

keyword = input("키워드 입력: ")
encText = urllib.parse.quote(keyword)
start_date = "2016-01-01"
end_date = str(datetime.now()-timedelta(days=1))[:10]
time_unit = "date"
keyword_groups = keyword
print(start_date, end_date)

키워드 입력: 전국운행 픽업 샌딩 콜벤 / 코로나19소독 방역콜밴 / 제이디 프리미엄 김포공항콜밴
2016-01-01 2021-02-15


In [4]:
def get_search_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    
    return {'Content-Type': 'application/json; charset=UTF-8', 
            'X-Timestamp': timestamp, 
            'X-API-KEY': API_KEY, 
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

In [5]:
def get_datalab_header(api_key, secret_key):
    return {
        "Content-Type":"application/json",
        "X-Naver-Client-Id": api_key,
        "X-Naver-Client-Secret":secret_key        
    }

In [6]:
BASE_URL = 'https://api.naver.com'
API_KEY = client_id
SECRET_KEY = client_secret
CUSTOMER_ID = customer_id

In [7]:
uri = '/keywordstool'
method = 'GET'
search_r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
                 params = {"sort":"date"},
                 headers=get_search_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

In [8]:
search_result = search_r.json()["keywordList"][0]
search_result

{'relKeyword': '김포공항',
 'monthlyPcQcCnt': 47000,
 'monthlyMobileQcCnt': 195200,
 'monthlyAvePcClkCnt': 13.4,
 'monthlyAveMobileClkCnt': 17.0,
 'monthlyAvePcCtr': 0.04,
 'monthlyAveMobileCtr': 0.01,
 'plAvgDepth': 12,
 'compIdx': '중간'}

In [9]:
import os
import sys
import urllib.request

In [43]:
client_id =  config_secret_debug["NAVER"]["CLIENT_ID"]
client_secret = config_secret_debug["NAVER"]["CLIENT_SECRET"]
url = "https://openapi.naver.com/v1/datalab/search"
body = { "startDate":start_date,
         "endDate":end_date,
         "timeUnit":time_unit,
         "keywordGroups" : [{"groupName": keyword, "keywords" : [keyword]}]
       }

In [44]:
datalab_r = requests.post(url,data = json.dumps(body),
                 headers=get_datalab_header(client_id, client_secret))

In [45]:
datalab_result = json.loads(datalab_r.content)
datalab_result

{'startDate': '2016-01-01',
 'endDate': '2021-02-15',
 'timeUnit': 'date',
 'results': [{'title': '김포공항',
   'keywords': ['김포공항'],
   'data': [{'period': '2016-01-01', 'ratio': 0.77537},
    {'period': '2016-01-02', 'ratio': 0.69934},
    {'period': '2016-01-03', 'ratio': 1.37163},
    {'period': '2016-01-04', 'ratio': 1.0909},
    {'period': '2016-01-05', 'ratio': 0.986},
    {'period': '2016-01-06', 'ratio': 0.94625},
    {'period': '2016-01-07', 'ratio': 0.95235},
    {'period': '2016-01-08', 'ratio': 0.98204},
    {'period': '2016-01-09', 'ratio': 0.9291},
    {'period': '2016-01-10', 'ratio': 0.91673},
    {'period': '2016-01-11', 'ratio': 1.05923},
    {'period': '2016-01-12', 'ratio': 1.12999},
    {'period': '2016-01-13', 'ratio': 1.22533},
    {'period': '2016-01-14', 'ratio': 1.02756},
    {'period': '2016-01-15', 'ratio': 1.60436},
    {'period': '2016-01-16', 'ratio': 0.97775},
    {'period': '2016-01-17', 'ratio': 0.92844},
    {'period': '2016-01-18', 'ratio': 1.3426},
  

### 최근 30일간 검색량 

In [20]:
import pandas as pd

In [21]:
month_ago = str(datetime.now()-relativedelta(months=1))[:10]
current_date = str(datetime.now())[:10]

In [22]:
dlab_list = datalab_result["results"][0]["data"]
month_index = next((index for (index, d) in enumerate(dlab_list) if d["period"] == month_ago), None)
dlab_list = dlab_list[month_index:]
ratio_sum = sum(dlab_list[i]["ratio"] for i in range(len(dlab_list)))
ratio_sum

40.151399999999995

In [23]:
print(str(datetime.now()-relativedelta(months=1))[:10])

2021-01-16


In [24]:
df = pd.DataFrame(search_result, index = ["values"])
df.drop(["compIdx",
         "plAvgDepth",
         "monthlyAveMobileCtr",
         "monthlyAvePcCtr",
         "monthlyAveMobileClkCnt",
         "monthlyAvePcClkCnt"], axis = 1 ,inplace = True)
df.rename(columns={"relKeyword":"검색어","monthlyPcQcCnt":"PC", "monthlyMobileQcCnt":"Mobile"}, inplace = True)
df["Total"] = search_result["monthlyPcQcCnt"] + search_result["monthlyMobileQcCnt"]
df["전월 대비"] = "_"

In [25]:
df["Total"] / ratio_sum

values    6032.168243
Name: Total, dtype: float64

In [26]:
display(df)

,검색어,PC,Mobile,Total,전월 대비
values,김포공항,47000,195200,242200,_


### 최근 컨텐츠 발행량

In [27]:
url_blog = "https://openapi.naver.com/v1/search/blog"
url_cafe = "https://openapi.naver.com/v1/search/cafearticle?query=" + encText

In [28]:
content_blog_r = requests.get(url_blog,
                              params = {"sort":"sim", "query" : encText, "display":100},
                              headers=get_datalab_header(client_id, client_secret))
content_cafe_r = requests.get(url_cafe,
                              headers=get_datalab_header(client_id, client_secret))

In [29]:
blog_result = json.loads(content_blog_r.content)
cafe_result = json.loads(content_cafe_r.content)

In [30]:
blog_result

{'lastBuildDate': 'Tue, 16 Feb 2021 14:17:21 +0900',
 'total': 97195,
 'start': 1,
 'display': 100,
 'items': [{'title': 'area2=%EB%82%A8<b>%EA%B</b><b>5%</b><b>AC</b>&amp;area3=%EC%9A%A<b>9%</b><b>ED%</b>98%B8%EB%<b>8F%</b>99',
   'link': 'https://blog.naver.com/wnstjd7942?Redirect=Log&logNo=222046640352',
   'description': 'https://m.r114.com/?_c=side&amp;_m=jun.search&amp;area1=%EB%B6%<b>80%</b>EC%82%B0<b>%EA%B</b>4%91%EC%97%<b>AD</b>%EC%8B%9C&amp;area2=%EB%82%A8<b>%EA%B</b><b>5%</b><b>AC</b>&amp;area3=%EC%9A%A<b>9%</b><b>ED%</b>98%B8%EB%<b>8F%</b>99',
   'bloggername': '주도적인 삶을 살아라',
   'bloggerlink': 'https://blog.naver.com/wnstjd7942',
   'postdate': '20200730'},
  {'title': '김포공항 주차팁 &quot;공항2임시공영주차장&quot;',
   'link': 'https://blog.naver.com/wjdalsrud17?Redirect=Log&logNo=222131670251',
   'description': 'top&amp;where=nexearch&amp;query=<b>%EA%B9%80%ED%8F%AC%EA%B3%B5%ED%95%AD</b>+%EC%A3%BC%EC%B0%A8%EC%9E%A5+%EC%9A%94%EA%B8%88&amp;oquery=<b>%EA%B9%80%ED%8F%AC%EA%B3%B5%ED%95%AD<

In [31]:
cafe_url = "https://cafe.naver.com/ca-fe/home/search/articles"+"?q="+encText+"&pr=3"
blog_url = "https://section.blog.naver.com/Search/Post.nhn"+"?pageNo=1"+"&rangeType=MONTH"+"&orderBy=sim"+"&startDate="+month_ago+"&endDate="+current_date+"&keyword="+encText

In [29]:
### Test url

import webbrowser
#webbrowser.open(blog_url)
#webbrowser.open(cafe_url)

Selenium + Django 어떻게 연결할 껀지 고민해보기

In [32]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import pathlib as p

In [33]:
## Selenium 속도 향상
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

In [34]:
driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")
driver.get(blog_url)
driver.implicitly_wait(3)
html = driver.page_source

In [35]:
soup = BeautifulSoup(html, "html.parser")
get_blog_number = soup.find("em",{"class","search_number"})
get_blog_number = int(get_blog_number.text[:-1].replace(",",""))
get_blog_number

8776

In [36]:
driver.get(cafe_url)
driver.implicitly_wait(3)

In [37]:
action = ActionChains(driver)
driver.find_element_by_id("app").click()
get_cafe_number = driver.find_element_by_class_name("total_count")
time.sleep(1)
get_cafe_number = int(get_cafe_number.text[:-1].replace(",",""))
print(get_cafe_number)

4611


In [38]:
driver.close()

In [39]:
df = pd.DataFrame(index = ["values"])
df["Blog"] = get_blog_number
df["Cafe"] = get_cafe_number
df["Total"] = get_blog_number + get_cafe_number
df["전월 대비"] = "_"
df

,Blog,Cafe,Total,전월 대비
values,8776,4611,13387,_


### Graph

In [53]:
## 카테고리에 있는 모든 카테고리 아이디 블러오기
naver_shop = "https://shopping.naver.com/"
driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")
driver.get(naver_shop)
time.sleep(1)

In [56]:
action = ActionChains(driver)
driver.find_element_by_class_name("co_category_menu").click()
category_list = driver.find_elements.by_tag_name("li")
print(category_list)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=88.0.4324.150)


In [38]:
#연령별 옵션 가능
age = input("연령: ")
#성별 옵션도 가능한데 일단은 보류

연령: 20


In [40]:
url = "https://openapi.naver.com/v1/datalab/shopping/categories";
body = { "startDate":start_date,
         "endDate":end_date,
         "timeUnit":time_unit,
         "category" : [
             {"name": "화장품/미용", "param": [ "50000002"]}
         ],
         "ages" : [age] 
       }

In [41]:
dlab_shopr = requests.post(url,data = json.dumps(body),
                 headers=get_datalab_header(client_id, client_secret))

In [42]:
dlab_shopr = json.loads(datalab_r.content)
dlab_shopr

{'startDate': '2016-01-01',
 'endDate': '2021-02-14',
 'timeUnit': 'date',
 'results': [{'title': '폼클렌징',
   'keywords': ['폼클렌징'],
   'data': [{'period': '2016-01-01', 'ratio': 47.66626},
    {'period': '2016-01-02', 'ratio': 46.98239},
    {'period': '2016-01-03', 'ratio': 48.16207},
    {'period': '2016-01-04', 'ratio': 44.94785},
    {'period': '2016-01-05', 'ratio': 40.29748},
    {'period': '2016-01-06', 'ratio': 45.28979},
    {'period': '2016-01-07', 'ratio': 43.34074},
    {'period': '2016-01-08', 'ratio': 43.30654},
    {'period': '2016-01-09', 'ratio': 44.75978},
    {'period': '2016-01-10', 'ratio': 43.06719},
    {'period': '2016-01-11', 'ratio': 48.53821},
    {'period': '2016-01-12', 'ratio': 49.94016},
    {'period': '2016-01-13', 'ratio': 45.85399},
    {'period': '2016-01-14', 'ratio': 48.53821},
    {'period': '2016-01-15', 'ratio': 50.2308},
    {'period': '2016-01-16', 'ratio': 44.70849},
    {'period': '2016-01-17', 'ratio': 47.56368},
    {'period': '2016-01-18', 

### 관련 키워드 현황

In [33]:
search_result

{'relKeyword': '폼클렌징',
 'monthlyPcQcCnt': 5940,
 'monthlyMobileQcCnt': 41200,
 'monthlyAvePcClkCnt': 13.2,
 'monthlyAveMobileClkCnt': 204.8,
 'monthlyAvePcCtr': 0.24,
 'monthlyAveMobileCtr': 0.53,
 'plAvgDepth': 15,
 'compIdx': '높음'}

In [38]:
df = pd.DataFrame(search_result, index = ["values"])
df.drop(["compIdx",
         "plAvgDepth"])


KeyError: "['compIdx' 'plAvgDepth'] not found in axis"

### 카테고리 인기 검색어

In [81]:
from selenium import webdriver
from time import sleep
from selenium.webdriver import ActionChains
import pyperclip
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import time
import win32com.client
import pandas as pd
import requests

### 예상 판매량

In [82]:
def get_category(item):
    categories = item.find_elements_by_tag_name("a")
    return [i.text for i in categories]

In [83]:
def get_link(item):
    link = item.find_element_by_tag_name("a").get_attribute("href")
    return link

In [84]:
def get_title(item):
    title = item.find_element_by_tag_name("a").text
    return title

In [85]:
def get_shipping(item):
    shipping_fee = 0
    fee = item.find_element_by_tag_name("em").text
    if not fee[4:].startswith("무료"):
        shipping_fee = fee[4:].replace("원","")
        shipping_fee = shipping_fee.replace(",","")
    return int(shipping_fee)

In [86]:
def get_store_name(item):
    store = item.find_element_by_class_name("basicList_mall__sbVax").text
    return store

In [87]:
def get_total_price(driver):
    get_buy_nums = driver.find_elements_by_class_name("Zye_YMu5OB")
    get_price = int(driver.find_element_by_class_name("_1LY7DqCnwR").text.replace(",",""))
    total = sum([int(i.text.split("건")[0]) for i in get_buy_nums])
    total_price = get_price * total
    return total_price

In [88]:
def get_sales(driver):
    get_buy_nums = driver.find_elements_by_class_name("Zye_YMu5OB")
    total = sum([int(i.text.split("건")[0]) for i in get_buy_nums])
    return total

In [89]:
def get_price(item):
    get_price = item.find_element_by_tag_name("span").text
    get_price = get_price.replace("원","")
    get_price = int(get_price.replace(",",""))
    return get_price

In [99]:
n_shop_url = "https://search.shopping.naver.com/search/all?frm=NVSHCHK&origQuery={}&pagingIndex=1&pagingSize=40&productSet=checkout&query={}&sort=rel&timestamp=&viewType=list".format(encText, encText)
driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")
driver.get(n_shop_url)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
time.sleep(1)

In [100]:
get_item_list = driver.find_element_by_class_name("list_basis")
items = get_item_list.find_elements_by_tag_name("div")
all_items_info = {}
info = {}
for item in items:
    class_name = item.get_attribute("class")
    if class_name.startswith("basicList_title"):
        title = get_title(item)
        info["link"] = get_link(item)
    elif class_name.startswith("basicList_mall_title"):
        info["store"] = get_store_name(item)
    elif class_name.startswith("basicList_depth"):
        info["category"] = get_category(item)
    elif class_name.startswith("basicList_mall_area"):
        info["shipping"] = get_shipping(item)
    elif class_name.startswith("basicList_price_area"):
        info["price"] = get_price(item)
    elif class_name.startswith("basicList_mall_grade"):
        info["price_shipping"] = info["shipping"] + info["price"]
        all_items_info[title]= info
        info = {}

In [101]:
for item in items:
    print(item.get_attribute("class"))



basicList_inner__eY_mq
basicList_img_area__a3NRA
thumbnail_thumb_wrap__1pEkS _wrapper
thumbnail_btn_box__3_V2T
thumbnail_btn__L5sLn
thumbnail_layer_info__2Bo5z
thumbnail_link_box__z5jwn
basicList_info_area__17Xyo
basicList_title__3P9Q7
basicList_price_area__1UXXR
basicList_depth__2QIie
basicList_desc__2-tko
basicList_detail_box__3ta3h
basicList_event__fLNNU
basicList_etc_box__1Jzg6
basicList_mall_area__lIA7R
basicList_mall_title__3MWFY
basicList_mall_grade__31CEX


In [102]:
for key,values in all_items_info.items():
    print(key)
    for title, value in values.items():
        print(title, value)
    print("------------------------")

전국운행 픽업 샌딩 콜벤 / 코로나19소독 방역콜밴 / 제이디 프리미엄 김포공항콜밴
link https://cr.shopping.naver.com/adcr.nhn?x=vyM8ceyywyzBTJM5%2F4roIP%2F%2F%2Fw%3D%3DsxFbVLtOgbz0SKacrKaS5o88s0hIXl9IwAzsRcPTfktfn36S%2BPozlL0oHlBkzL%2BGwWF3qRqDkUe%2B8pRmTWAdRDgukGVyLnwV1C1xZer7oGSn8mtz%2FeZHSwjJ42cOwj0Cfhwi%2Bso400G4fayQzDpG0EOTNFG8xSlnsEaHGgiisktaIU2nTQRckpgMldaETzgSPUJdIgSYnosQraYLJ%2FMuKUZXeiXbxuiD%2F8EOfb7uK9qhqZnDVwMmLkCFFKcmu4TDDFe00g%2BKOZqM0JekrlAxQDkS%2BortKzfQDWBF5GV%2FKfqEhJv3ljJo79VVIpj6%2Fm3WOkXjkJnky%2FB7WMGUsk8jI%2Bgcyz48mW%2B%2BYu%2BCClhVFVTxjI50HZy%2FcLV2dSyPQS0HhEIZJB1yuP5kkcuYnGBP9u%2B9ntzLRX7%2FckGw2yxcGq%2BP1M2d1CXZt2h8Jd2SRb4JfaXEPSTSJxSd4DjaxOs7a3NEJeUCmpPKPpNh7yzUHda8E7G6ucT0VhBDFUuE4TRg0ncDKdxLGvM44%2FNfX0QkIskj36agcF%2FJ%2BnxUGI9gn%2FMbDhGvU%2FlCgT%2BZpmbfQNx8K5ecsr6Dgwc4e%2FHGtdzEJowAClQzNqN%2BEXPeM244U5rNr8sz1Ni%2FKXUa6hhmVYKKL%2BzdVAtahD3v4497z0IU1aAkk4r4sLJIl2oHt%2FF6Bv3vQ2JFnEOKnhePRU54S2FpUbY9HZaG4rt4S8eJjMyqSRWVpY4ojQ%2FmQYfDbR2hEK3224HzMFPCAgbu1hTl8ArfHqk9k3UaW6Ri3v%2By6YPmuAm9TZjqEQTAb

In [103]:
for title,info in all_items_info.items():
    begin = time.time()
    driver.get(info["link"])
    time.sleep(1)
    try:      
        sales = get_sales(driver)
    except:
        sales = "Unknown"
        print("Time elapsed: {:.3f}s".format(time.time() - begin))
    info["sales"] = sales
    all_items_info[title] = info

In [104]:
#driver.close()
for key, values in all_items_info.items():
    print(key, values["price"])

전국운행 픽업 샌딩 콜벤 / 코로나19소독 방역콜밴 / 제이디 프리미엄 김포공항콜밴 40000


In [105]:
from collections import OrderedDict

ordered = OrderedDict(sorted(all_items_info.items(), key=lambda i: i[1]["sales"], reverse = True))

In [106]:
for key,values in ordered.items():
    print(key, values["sales"])

전국운행 픽업 샌딩 콜벤 / 코로나19소독 방역콜밴 / 제이디 프리미엄 김포공항콜밴 0


In [107]:
driver.close()